<img src="../Images/DSC_Logo.png" style="width: 400px;">

# Time Series Theory in Python - Part 3: Time Series Preprocessing

This notebook presents several techniques for time series preprocessing, which are crucial for preparing data for analysis and predictive modeling. These techniques can serve as independent analyses on their own and facilitate further preprocessing tailored to specific prediction tasks.

In [ ]:
pip install statsmodels PythonTsa statsmodels

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## 1. Smoothing

### **Example 1: Australian Employed Total Persons**

In [ ]:
# Get the data file path
dtapath = getdtapath()

# Load the Excel file containing employment data
aul = pd.read_excel(dtapath + 'AustraliaEmployedTotalPersons.xlsx', header=0)

# Apply smoothing using a moving average
window_size = 12  # Define the window size for moving average (e.g., 12 for yearly smoothing)
y_smoothed = pd.Series(y).rolling(window=window_size, center=True).mean().values

# Plotting the results
plt.figure()
plt.scatter(aul.index, y, label='Original Data (Total Employed Persons)', color='gray', alpha=0.5)  # Original data
plt.plot(aul.index, y_smoothed, label='Smoothed Data (Moving Average)', color='blue', linewidth=2)  # Smoothed data

# Adding titles and labels
plt.title('Original and Smoothed Number of Employed Persons in Australia')
plt.xlabel('Date')
plt.ylabel('Number of Persons Employed')  # Clarified y-axis label
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

## 2. Detrending

The resulting detrended data represents the deviations from the trend. The values in the detrended series indicate how much each observation is above or below the trend line. 

### **Example 2: Mean Spring Passage Dates of European Pied Flycatcher**

The dataset contains the migration data of the European Pied Flycatcher, focusing on the adjusted mean spring passage dates (MADJDAYSWS) across multiple years. The dataset includes annual observations.

**Original dataset and code:** Haest, B., Hüppop, O., & Bairlein, F. (2020). Code and data for: "Weather at the winter and stopover areas determines spring migration onset, progress, and advancements in Afro-Palearctic migrant birds". In Proceedings of the National Academy of Sciences of the United States of America (v1.0, Bd. 117, Nummer 29, S. 17056–17062). Zenodo. doi:10.5281/zenodo.3629178              

**Related publication(s):** Haest, B., Hüppop, O., and Bairlein, F.: Weather at the winter and stopover areas determines spring migration onset, progress, and advancements in Afro-Palearctic migrant birds, Proceedings of the National Academy of Sciences, 117, 17056–17062, doi:10.1073/pnas.1920448117,              2020.

Original data and code were modified for this notebook.

In [ ]:
bird_migration = pd.read_csv('../Datasets/bird_migration.csv', sep = ';')

In [ ]:
import statsmodels.api as sm

# Assuming the 'MADJDAYSWS' is the column you want as y
y2 = bird_migration['MADJDAYSWS'].values  # Extracting dependent variable
time2 = bird_migration['Year'].values  # Using the Year column as the time variable

# Checking the lengths
print(f'Length of y: {len(y2)}')
print(f'Length of time: {len(time2)}')

linear_model = sm.OLS(y2, sm.add_constant(time2)).fit()

# Perform linear detrending
trend = linear_model.fittedvalues  # This is the linear trend
detrended_y = y2 - trend

# Plot
plt.figure()
plt.plot(time2, y2, label='Original Data', color='gray', alpha=0.5)  # Original data
plt.plot(time2, trend, label='Linear Trend', color='blue', linewidth=1.5)  # Linear trend
plt.plot(time2, detrended_y, label='Detrended Data', color='red', linewidth=1.5)
plt.title('Original and Detrended MADJDAYSWS')
plt.xlabel('Year')
plt.ylabel('MADJDAYSWS')  # Optionally change to 'Detrended MADJDAYSWS'
plt.legend()
plt.tight_layout()
plt.grid()
plt.show()

## 3 Scaling

In [ ]:
# prepare data for normalization
values = y2.reshape((len(y2), 1))

(Reshaping on-dimensional data into a two-dimensional format before scaling is necessary because most preprocessing functions expect input data to have the shape of (n_samples, n_features). This ensures that the data is correctly interpreted as multiple samples of one or more features, allowing for proper scaling and processing during model training.)

## 3.1 Normalization

$$y = (x - min) / (max - min)$$

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# train the normalization
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(values)

# normalize the dataset and print the first 5 rows
normalized = scaler.transform(values)
print(normalized[:5])

# inverse transform and print the first 5 rows
inversed = scaler.inverse_transform(normalized)
print(inversed[:5])

In [ ]:
print("Min value normalized:", min(normalized))
print("Max value normalized:", max(normalized))

## 3.2 Standardization

$$y = (x - mean) / std$$

In [ ]:
from sklearn.preprocessing import StandardScaler
from math import sqrt

# train the standardization
scaler = StandardScaler()
scaler = scaler.fit(values)

# standardization the dataset and print the first 5 rows
normalized = scaler.transform(values)
print(normalized[:5])
    
# inverse transform and print the first 5 rows
inversed = scaler.inverse_transform(normalized)
print(inversed[:5])

In [ ]:
print("Min value normalized:", min(normalized))
print("Max value normalized:", max(normalized))

## 4. Outlier/Anomaly Detection

### **Example 3: Artificial Streamflow Time Series with Outliers and Change Point**

We create an artificial time series to simulate the dynamics of streamflow over a defined period while incorporating features like outliers and a change point. 

In [ ]:
# Parameters
np.random.seed(42)  # For reproducibility
days = pd.date_range(start='2021-01-01', end='2023-12-31', freq='D')
n = len(days)

# Generate a normal streamflow pattern (in m³/s)
streamflow = 10 + 0.5 * np.sin(np.linspace(0, 20 * np.pi, n)) + np.random.normal(0, 1, n)

# Introduce a change point after 1.5 years (midway through the second year)
change_point_index = int(n * 0.75)
streamflow[change_point_index:] *= 0.5  # Effect of drying out

# Introduce a total of 7 random outliers
outliers_indices = np.random.choice(range(n), size=7, replace=False)
streamflow[outliers_indices] += np.random.normal(10, 5, size=7)  # Adding outliers above mean

# Create a DataFrame
data = pd.DataFrame(data={'Date': days, 'Streamflow (m³/s)': streamflow})

# Plot the time series
plt.figure()
plt.plot(data['Date'], data['Streamflow (m³/s)'], label='Streamflow', color='b')
plt.axvline(x=data['Date'][change_point_index], color='r', linestyle='--', label='Change Point')
plt.plot(data['Date'][outliers_indices], data['Streamflow (m³/s)'][outliers_indices], 'ro', label='Outliers')  # Mark all outliers in red
plt.title('Artificial Streamflow Time Series with Outliers and Change Point')
plt.xlabel('Date')
plt.ylabel('Streamflow (m³/s)')
plt.legend()
plt.grid()
plt.show()

## 4.1 Z-Score

In [ ]:
mean = np.mean(streamflow)
std_dev = np.std(streamflow)
z_scores = (streamflow - mean) / std_dev
outliers_z = np.where(np.abs(z_scores) > 3)[0]

plt.figure()
plt.plot(data['Date'], data['Streamflow (m³/s)'], label='Streamflow', color='b')
plt.scatter(data['Date'][outliers_z], streamflow[outliers_z], color='red', label='Z-score Outliers', zorder=5)
plt.title('Z-Score Method for Outlier Detection')
plt.xlabel('Date')
plt.ylabel('Streamflow (m³/s)')
plt.legend()
plt.grid()

## 4.2 Moving Average Method

In [ ]:
moving_average = pd.Series(streamflow).rolling(window=7).mean()
differences = np.abs(streamflow - moving_average)
outlier_threshold = differences.mean() + 3 * differences.std()  # Define threshold for outliers
outliers_moving_avg = np.where(differences > outlier_threshold)[0]

plt.figure()
plt.plot(data['Date'], streamflow, label='Streamflow', color='b')
plt.plot(data['Date'], moving_average, label='7-Day Moving Average', color='orange')
plt.scatter(data['Date'][outliers_moving_avg], streamflow[outliers_moving_avg], color='red', label='Moving Avg Outliers', zorder=5)
plt.title('Moving Average Method for Outlier Detection')
plt.xlabel('Date')
plt.ylabel('Streamflow (m³/s)')
plt.legend()
plt.grid()

## 4.3 Density-based Methods: DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
# DBSCAN Outlier Detection
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data[['Streamflow (m³/s)']])  # Scale the data

# Fit DBSCAN (adjust eps and min_samples as needed)
dbscan = DBSCAN(eps=0.5, min_samples=10)  # eps controls the neighborhood radius, and min_samples controls density threshold
data['Outlier'] = dbscan.fit_predict(data_scaled)

# DBSCAN labels outliers as -1, so we mark them
dbscan_outliers = data[data['Outlier'] == -1]

# Plot the time series with outliers
plt.figure()
plt.plot(data['Date'], data['Streamflow (m³/s)'], label='Streamflow', color='b')
plt.plot(dbscan_outliers['Date'], dbscan_outliers['Streamflow (m³/s)'], 'ro', label='DBSCAN Detected Outliers')
plt.title('Artificial Streamflow Time Series with Outliers and Change Point (DBSCAN Detection)')
plt.xlabel('Date')
plt.ylabel('Streamflow (m³/s)')
plt.legend()
plt.grid()
plt.show()

## 5. Data Availability and Gaps

## 5.1 Data Availability in Time Series

### **Example 4: Large Dataset of Synthetic Time Series with Gaps**

We generate multiple synthetic independent time series with trend, seasonal effect, random noise AND gaps.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic time series data
dates = pd.date_range('2000-01-01', periods=100, freq='M')

# Create a DataFrame to hold the time series
data = pd.DataFrame(index=dates)

# Generate time series
n_series = 10
for i in range(1, n_series + 1):  # 24 time series
    trend = np.linspace(0, np.random.uniform(0.1, 1), len(dates))  # Linear trend
    seasonal = 0.1 * np.sin(np.linspace(0, 3 * np.pi, len(dates)))  # Seasonal effect
    noise = np.random.normal(loc=0, scale=0.05, size=len(dates))  # Random noise
    series = trend + seasonal + noise  # Combine to create the time series
    data[f'Series_{i}'] = series

# Add gaps (missing values) to the series
num_gaps = 10  # Number of NaN values to insert in each series
for column in data.columns:
    indices = np.random.choice(data.index, num_gaps, replace=False)  # Select random indices
    data.loc[indices, column] = np.nan  # Assign NaN values

# Plotting all the time series; highlight the last two series
plt.figure()
for column in data.columns:
    plt.plot(data.index, data[column], label=column, color='blue')
plt.title('Synthetic Time Series Data with Gaps')
plt.xlabel('Date')
plt.ylabel('Values')
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# prepare data for heatmap
heatmap_data = data.notna().astype(int)  # Convert to binary (1 for available, 0 for NaN)
heatmap_data = heatmap_data.T
print(heatmap_data.head())

# plot heatmap
import seaborn as sns
plt.figure()  
sns.heatmap(availability_data.T, cmap='binary', cbar_kws={'label': 'Data Availability'}, annot=False)
plt.title("Heatmap of Data Availability by Sample ID and Date")
plt.xlabel("Sample ID")
plt.ylabel("Date")
plt.tight_layout()
plt.show()

## 5.2 Interpolation

In [ ]:
# Fill the gaps using linear interpolation
data_filled = data.interpolate(method='linear')

# Plotting the original data and the filled data
plt.figure()
plt.plot(data_filled, label='Data after Filling', color='orange')
plt.plot(data, label='Original Data', color='b')
plt.title('Data with Filling Gaps')
plt.xlabel('Date')
plt.ylabel('Values')
handles, labels = plt.gca().get_legend_handles_labels() # Create handles for the legend manually
by_label = dict(zip(labels, handles)) # Remove duplicates by turning the labels into a set and getting unique corresponding handles
plt.legend(by_label.values(), by_label.keys()) # Create the legend with unique entries
plt.grid()
plt.show()

### **Example 3 [continued]: Artificial Streamflow Time Series with Outliers and Change Point**

In [ ]:
# Remove outliers from the data
filtered_streamflow = data['Streamflow (m³/s)'].copy()
filtered_streamflow[outliers_moving_avg] = np.nan  # Set outlier points to NaN

# Fill the gaps using linear interpolation
filled_streamflow = filtered_streamflow.interpolate(method='linear')

# Plotting the original data and the filled data
plt.figure()
plt.plot(data['Date'], data['Streamflow (m³/s)'], label='Original Streamflow', color='b')
plt.plot(data['Date'], filled_streamflow, label='Streamflow after Outlier Removal and Filling', color='orange')
plt.scatter(data['Date'][outliers_moving_avg], streamflow[outliers_moving_avg], color='red', label='Removed Outliers', zorder=5)
plt.title('Streamflow Data with Outlier Removal and Filling Gaps')
plt.xlabel('Date')
plt.ylabel('Streamflow (m³/s)')
plt.legend()
plt.grid()
plt.show()

# Zooming in on one of the removed outlier points (e.g., using the first outlier)
outlier_to_zoom = outliers_moving_avg[0]  # Choose an example outlier to zoom in
start_date = data['Date'][outlier_to_zoom - 5]  # 5 days before
end_date = data['Date'][outlier_to_zoom + 5]    # 5 days after

# zoomed-in
plt.figure()
plt.plot(data['Date'], data['Streamflow (m³/s)'], label='Original Streamflow', color='b', alpha=0.5)
plt.plot(data['Date'], filled_streamflow, label='Streamflow after Outlier Removal and Filling', color='orange')
plt.scatter(data['Date'][outliers_moving_avg], streamflow[outliers_moving_avg], color='red', label='Removed Outliers', zorder=5)
plt.xlim([start_date, end_date])
plt.title('Zoomed In View Around Removed Outlier')
plt.xlabel('Date')
plt.ylabel('Streamflow (m³/s)')
plt.legend()
plt.grid()
plt.show()

## 5.3 Reindexing

### **Example 4 [continued]: Large Dataset of Synthetic Time Series with Gaps**

## 5.3.1 Resampling

For aggregating existing values over defined time interval.

In [ ]:
# Use resample to get yearly mean values
yearly_results = data.resample('Y').mean()

# Example of accessing the yearly aggregated data
print(yearly_results)

# Plotting the original data and the filled data
plt.figure()
plt.plot(yearly_results, label='Original Data', color='b')
plt.title('Yearly aggregated data')
plt.xlabel('Date')
plt.ylabel('Values')
plt.grid()
plt.tight_layout()
plt.show()

## 5.3.2 Interpolate & Select By Sequence

Focusing on estimating values at specific times where they might not exist.

In [ ]:
series_data.head()

In [ ]:
from scipy.interpolate import interp1d

# Sequence for interpolation (years from 2000 to 2009)
seq_interpol = np.arange(2000, 2010, 1)

# Create a new DataFrame for storing interpolated results
yearly_results = pd.DataFrame(index=seq_interpol)

# Interpolation for each column in the data DataFrame
for column in data.columns:
    
    # Reset index to access Year easily for each specific series
    series_data = data[column].reset_index()  # Create series_data inside the loop

    # Rename columns for clarity
    series_data.columns = ['Date', 'Values']

    # Convert Date to its years
    series_data['Year'] = series_data['Date'].dt.year
    
    # 1. Drop NaN values for interpolation
    series_data = series_data.dropna(subset=['Values'])
    
    # 2. Interpolate using linear interpolation
    # Ensure to use the 'Year' for interpolation
    f = interp1d(series_data['Year'], series_data['Values'], 
                 kind='linear', bounds_error=False, fill_value=np.nan)
    
    # 3. Get interpolated values for the predefined sequence
    interpolated_values = f(seq_interpol)

    # Store interpolated values in the yearly results DataFrame
    yearly_results[column] = interpolated_values

# Example of accessing the interpolated data
print(yearly_results)

# Plotting the original data and the filled data
plt.figure()
plt.plot(yearly_results, label='Original Data', color='b')
plt.title('Yearly aggregated data')
plt.xlabel('Date')
plt.ylabel('Values')
plt.grid()
plt.tight_layout()
plt.show()